Helmiina Hotti, University of Helsinki, Machine Learning for Linguists 2019


# Genre Classification with Vectors


## Introduction

In this report, I will guide you through my solution to the problem of classifying documents into genres. I will try two different solutions for this task: first unsupervised learning with a K-Means clusterer (part 1) and then supervised learning (part 2). The task is intriguing, as it is a great test for how accurately word meaning can be captured in arrays of numbers, aka vectors. It also tests the suitability of a K-Means clusterer for a task like this. The results of this project will also provide us with information regarding the genres in question: we will find out what genres are similar to each other when it comes to contents, and which ones are easily separable from each other. This actually creates the biggest challenge for our experiment; how well can our models learn to separate genres that are somewhat similar to each other. 

We will walk through the code step by step, while looking at why what we are doing is relevant and necessary. I will also try my best to provide you with information regarding the theory of clustering with a K-Means clusterer, unsupervised vs. supervised learning and vector classification.


## Part 1: Unsupervised learning


### 1.1 Data and Tools

First we need to import the needed tools and data for this project. For mathematical calculations we will need the math and numpy libraries, which come with a wide selection of useful tools. In order to create the K-Means clusterer for our unsupervised solution, we need the KMeansClusterer and cosine_distance imports from NLTK. For randomizing data and vectors, we will use the random-library. Finally, for the actual data we will use the Brown corpus from NLTK. 

In [ ]:
import math
import random
import numpy
from nltk.cluster import KMeansClusterer, euclidean_distance
from nltk.corpus import brown

Next we are going to narrow down our data to seven genres of texts. The chosen genres are news, religion, romance, science fiction, government, mystery and humor. The total amount of files in the chosen categories is 159, with 373426 word tokens and 25269 word types. In order to obtain valid results, want our vocabulary to be uniform and normalized to lower-case, which is done when determining the vocabulary. 

In [ ]:
categories = ['news', 'religion', 'romance', 'science_fiction', 'government', 'mystery', 'humor']
text = [w for w in brown.words(categories=categories)]
vocabulary = set([word.lower() for word in text])
documents = brown.fileids(categories=categories)

### 1.2 Method

As mentioned, the chosen method for classifying genres in the first part of this project is unsupervised learning with vectors. Unsupervised learning means essentially that the machine learning model does not have access to the correct labels of the data it is classifying during 'learning', but it works more independently. This differs from supervised learning in the sense that the data is not split into training and testing sets, due to the fact that the model does not learn from the gold standard, but the model has access to the entirety of the data at once, and uses alternative methods to classify it (in our case, a clusterer). More information about supervised learning will be given in part 2. The method used here is creating vectors that eventually contain 'meaning', and letting a K-Means clusterer attempt to classify the documents into as many categories as we have genres, hoping that the clusters will match the genres.

Evaluation will be conducted by comparing the contents of the clusters produced by our clusterer to the contents of the genres in the Brown corpus, our gold standard.


### 1.2.1 Initializing and training vectors

First we need to initialize two types of vectors. These are index vectors that represent words in the vocabulary and context vectors that represent entire documents. The dimensionality of the vectors is set to 200 and all values in the vectors are initialized to 0. To implement random mapping (by Samuel Kaski, 1998), we will randomly set five values from each index vector to 1. Random mapping allows us to get sufficient document classification accuracies with only a vector dimensionality of 200. In order to build a vector model with a decent accuracy without random mapping, a dimensionality of several hundred if not thousand would be needed, which is time-consuming and costly in real-life research situations. 

In [ ]:
d = 200
m = 5

index_vector = {word: [0]*d for word in vocabulary}
context_vector = {doc: [0.0]*d for doc in documents}

for word in vocabulary:
    random_positions = list(range(0, d))
    random.shuffle(random_positions)
    for i in random_positions[:m]:
        index_vector[word][i] = 1

Next we need a function that sums together wanted vectors. This gives our context vectors the 'context'. How this works is we go through all documents in our data, collect the words (normalized to lower-case) in the document at hand and add together the context vector for the document and the index vector for the word. This creates a unique vector for each document, which can then be utilized in classifying the documents into clusters.

In [ ]:
def add_vector(a, b):
    for i, x in enumerate(b):
        a[i] += x
        
for doc in documents:
    context = [w.lower() for w in (brown.words(doc))]
    for word in context:
        add_vector(context_vector[doc], index_vector[word])

### 1.2.2 Preparing needed mathematical functions

In order to not let word frequencies alter the valuable information the vectors hold, the vectors need to be normalized. In order to do that, we need a function that calculates vector length. The normalizing itself is done by the function normalize(). 

In [ ]:
def vector_length(v):
    total = sum(x ** 2 for x in v)
    squared = math.sqrt(total)
    return squared

def normalize(a):
    return [(x / vector_length(a)) for x in a]

### 1.2.3 Clustering

As mentioned in the introduction, our unsupervised classification is done with a K-Means clusterer. K-Means clusterer is one of the most commonly used clusterers. In short, the clusterer works by first randomly dividing data points into the wanted number of clusters, computing the centroid (the average of the vectors in the cluster) of each cluster and reassigning the clusters so that each vectors belongs to the cluster represented by the closest centroid. This two-step procedure is repeated for wanted amount of times; the number of repeats is determined when initializing the clusterer. The final set of clusters is the result of the clusterer.

Below, we first create the list 'vectors' that consists of normalized context vectors for each document in our data set. The number of clusters is set to seven, as we have seven genres. Then we initialize the clusterer so it produces the wanted amount of clusters, uses euclidean distance (the distance of vectors in the euclidean space) to calculate similarities and repeats the procedure described above 10 times. In order to print the clusters somewhat nicely, we zip together the documents and their determined cluster and assign them into lists based on the cluster they belong to. Finally we can print the results of our clusterer as well as the gold standard for evaluation. 

In [ ]:
# Determining the vectors the clusterer will cluster:
vectors = [numpy.array(f) for f in [normalize(context_vector[d]) for d in documents]]
# Determining the number of clusters:
n_clusters = 7
# Initializing clusterer and clustering:
clusterer = KMeansClusterer(n_clusters, euclidean_distance, repeats=10)
clusters = clusterer.cluster(vectors, assign_clusters=True, trace=False)
zipped = sorted(list(zip(documents, clusters)), key=lambda x: x[1])

i = 0
list_all = []
while i <= max(clusters):
    list_c = []
    for item in zipped:
        if item[1] == i:
            list_c.append(item)
    list_all.append(list_c)
    i += 1

# Printing results:
print("Clusters:\n")
for i in list_all:
    print("Cluster #" + str(i[0][1]) + ": " + ", ".join(w for w, c in i))

# Printing the gold standard:
print("\nGold standard:\n")
for category in categories:
    print(category + ":", ", ".join(brown.fileids(categories=category)))

### 1.3 Results and discussion

Above we see the results of our clusterer. In the Brown corpus, the genre of each file is visible in the two letters of the file id: 

ca = news, cd = religion, ch = government, cl = mystery, cm = science fiction, cp = romance, cr = humor

As we can see from the clusters, the performance is quite lackluster. What we wanted to see was six clusters, with only files that belong to the same genre in each, like what we see under the Gold standard -header. There are, however, little to no clusters with a clear majority of any specific category, but most clusters are a mix of three or more genres (this varies from run to run so it is not meaningful for me to write the specifics here). 

It is really difficult to calculate the accuracy of the clusterer, because it does not know how many items there should be in each category, so comparing the performance to the gold standard is really hard. If you were to determine the genre of a cluster by taking the genre that is the most prominent in that cluster, you would end up with no science fiction category at all, as the six files of that genre form the minority in each cluster that they appear in. I looked into and tried calculating the purity of the clusters, but I could not get past the problem of determining the genre of a cluster where there is no clear majority of any genre. Therefore, we need to settle with just looking at the clusters and evaluating the performance that way.

I think the fact that the amount of documents varies so much between the genres creates a big problem not only for the evaluation but to the performance itself. The clusterer does not know that we need a cluster with only six files as well as a cluster with 44 files, which probably confuses the clusterer. I also think that the vocabularies of each genre are not different enough from each other, which makes clustering harder. I attempted to improve the performance by removing stop words from the data, in order to hopefully make the vocabularies of each genre more distinct. Doing that, however, made the program so slow that it was unusable (each run would take over 30 minutes).

In the next part of the report, we will look at a supervised solution for the same task.

## Part 2: Supervised learning

### 2.1 Data and tools

The data for this experiment is the same for this experiment than the first one. However, we will now utilize the genres as well as the documents in our data. The data will be divided in half: the first 30% is used for testing and the rest (70%) is used for training. Setting the context for our genre vectors will happen a bit later, but when determining our test set below we already extract only the first half of documents in each genre and add them to our test set called test_documents.

In [ ]:
all_genres = brown.categories()

train_genres = []
for genre in all_genres:
    if genre in categories:
        train_genres.append(genre)

test_documents = []
for genre in train_genres:
    size = int(len(brown.fileids(categories=genre)) * 0.3)
    for file in brown.fileids(categories=genre)[:size]:
        test_documents.append(file)

### 2.2 Method

As mentioned, the method used in this part of the project is supervised vector learning. This differs from unsupervised learning in that the model is trained with a training set, which consists of the context vectors of our genres into which we will finally classify our documents, whereas in unsupervised training, the model never sees the categories into which it classifies, and their contents. Here, training provides the model information of the contents of the genres, and it utilizes that knowledge when assigning the documents into genres; it looks at the contents of each document vector and finds a genre that is closest to it when it comes to content. The data for training and testing sets are disjoint, so testing is not conducted using the data that the model is already familiar with, but it will have to utilize the knowledge it gathered during training when classifying the test set. 


### 2.2.1 Creating and training new vectors

Below we initialize our context vectors for documents and genres (separately). Again, like in the first part, we train our context vectors by adding the context vectors together with the index vectors for each word in the set. We already have index vectors for every word in our vocabulary, as we did that in the first part, so there is no need to alter them in any way. The dimensionality of the vectors (d) will be the same as in the first part.

When training the genre vectors, we will only utilize 70% of the documents in each genre. This is because we will use the first 30% in testing, and training and testing sets need to be kept separate. 

In [ ]:
# Initializing document and genre vectors and setting the dimensionality to 200:
context_vector_document = {doc: [0.0] * d for doc in test_documents}
context_vector_genre = {genre: [0.0] * d for genre in train_genres}

# Training document context vectors:
for doc in test_documents:
    context = [w.lower() for w in (brown.words(doc))]
    for word in context:
        add_vector(context_vector_document[doc], index_vector[word])

# Training genre vectors with the last half of the documents in each genre:
for genre in train_genres:
    size = int(len(brown.fileids(categories=genre)) * 0.7)
    context = [w.lower() for w in [w for w in brown.words(brown.fileids(categories=genre)[size:])]]
    for word in context:
        add_vector(context_vector_genre[genre], index_vector[word])

### 2.2.2 Preparing more mathematical functions

In the comparison and classification of documents and genres, we will utilize cosine distance. Cosine distance is the cosine of the angle between two vectors, and it is a great way to calculate the similarity of them. In order to calculate the cosine distance, we need to first calculate the length of the vectors we are comparing. As it happens, we also utilize vector length in our vector normalization function, so we have already created a function for that. Now we just need a function that calculates the cosine distance. It takes as parameters the two vectors of which cosine distance we wish to calculate. 

In [ ]:
def cosine_dist(a, b):
    total = 0
    i = 0
    while i <= (len(b) - 1):
        total += a[i] * b[i]
        i += 1
    return 1 - (total / (vector_length(a) * vector_length(b)))

### 2.3 Calculating cosine distances between documents and genres

Predicting the genre of each document in our test set takes place in the function pairwise_distance() below. The function takes as parameters the set of documents we wish to classify (our test set) and the set of possible genres (our training set). The function goes through all documents and genres, and saves every possible document-genre pair, together with their cosine distance, to a list. This is needed in order to find the pair with the lowest cosine distance in the next step.

Next, the function goes through all documents, and for each document it picks the items from the list of pairs and distances that include that document. Then it looks for the document-genre pair with the lowest cosine distance, and the genre of that pair is the predicted genre. The pair is then added to the list min_pairs, which will eventually include all documents with their predicted genre and their cosine distance. If the genre with the lowest cosine distance matches the gold standard, the varible 'correct' is increased by 1. Correspondingly, if the predicted genre does not match the gold standard, the variable 'incorrect' is increased by one. This is how we keep track of the accuracy of the model.

Now that we have the results stored in min_pairs, we can print them. The results will be printed tidily under headers, sorted alphabetically by the document id.

In [ ]:
def pairwise_distance(docs, genres):
    print_pairs = []
    min_pairs = []
    correct = 0
    incorrect = 0
    
    for doc in docs:
        for genre in genres:
            # Adding every possible document-genre pair together with their cosine distance to a list:
            print_pairs.append((doc, genre, cosine_dist(
                normalize(context_vector_document[doc]),
                normalize(context_vector_genre[genre]))))
            
    for doc in docs:
        # Looking for pairs that include the document at hand: 
        relevant = [pair for pair in print_pairs if pair[0] == doc]
        # Finding the document-genre pair with the smallest cosine distance and adding it to min_pairs:
        min_distance = min(relevant, key=lambda d: d[2])
        min_pairs.append(min_distance)
        # Keeping track of performance:
        if min_distance[1] == brown.categories(doc)[0]:
            correct += 1
        else:
            incorrect += 1
            
    # Printing the table of results sorted by file id:
    print('%-10s %-17s %-17s %-15s' % ("File id", "Predicted genre", "Cosine similarity", "Gold standard"))
    for i in sorted(min_pairs, key=lambda x: x[0]):
        print('%-10s %-17s %-17.3f %-15s' % (i[0], i[1], i[2], brown.categories(i[0])[0]))
    print()
    
    # Printing statistics:
    print("Correctly classified documents:", correct, "(" + str(float(correct) / len(test_documents) * 100) + "%)")
    print("Incorrectly classified documents:", incorrect)

### 2.4 Results and discussion

Running the next block of code prints us our results. As mentioned in the results section of the first part, the genre of each file is visible in the file id: those beginning with 'ca' are labeled 'news', 'cd' means 'religion', 'ch' stands for 'government', 'cl' stands for 'mystery', 'cm' means science fiction, 'cp' stands for romance and 'cr' for humor. The correct genre is also printed in the 'Gold standard' column. 

The accuracy of the model varies between around 79-87%. There seems to be a pattern when it comes to what the model gets confused with, even though the results differ quite a bit from run to run. The genres science fiction, mystery and romance seem to get mixed up rather often, which is not all that surprising, considering that they all are genres of prose, and probably have quite similar vocabularies. Sometimes it also misclassifies some government texts for religion. There is little (but sometimes some) mismatch in the other genres. It also seems like the genres with the most documents are classified correctly most successfully; which makes sense, as there is more data to train with. Correspondingly, the model seems to never classify correctly the one science fiction document that is in the test data, as there is only five documents to train with. Overall, the performance is quite good, and it always gets more documents correct than incorrect in every genre, except for science fiction, where there is only one document to classify. 

All in all, I would say that this supervised learning method is suitable for this kind of a task. The model is efficient, simple and runs quick, and the accuracy is quite good. However, like many other classification methods, this model is prone to mistakes with genres and documents that are similar to each other. The performance is absolutely not perfect, and a more advanced model, which tackles the issue of similarity of vocabularies, would be more suitable for real-life research or work situations, where there are more genres and documents in the data. 

In [ ]:
pairwise_distance(test_documents, train_genres)

## Conclusions

In this report we looked at classifying documents into genres with two different approaches: unsupervised learning and supervised learning. The method for unsupervised learning was a K-Means clusterer and for supervised learning we utilized cosine distance comparisons of document vectors and genre vectors. As we saw from the results sections of each part, the supervised learning approach worked considerably better for the task. We managed to get classfication accuracies between 79 and 87%, while the unsupervised method worked considerably worse (which was evident even though we could not conduct proper evaluation). 

We identified (and assumed) two types of problems for the clusterer: the difference in the number of documents between genres and the similarity of vocabularies between genres. The latter was also an issue for our supervised method, but the first one did not create that big of a problem, except for the fact that genres with only a few documents were harder to predict. We came to the conclusion that in order to get the accuracy even better, we would need a way to reflect the more subtle differences in vocabularies onto our context vectors in order to better differentiate between genres that are close to each other in content, such as the prose genres 'mystery' and 'romance'. What would also improve the performance would be having more consistent amounts of data per genre; now we have 44 documents of 'news' texts and only six of 'science fiction', which means in practice that the training set for 'news' is considerably larger than for 'science fiction', and what follows is that the model can learn better about the genre with more data. 

Even though the performance of the supervised model we created is good, it is not a perfect solution by any means, and a more advanced model would be needed in real-life research and work situations. This was, however, a successful experiment in my opinion. 